In [5]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report

# Load dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
x_train, x_test = np.expand_dims(x_train, -1), np.expand_dims(x_test, -1)



def build_siamese_model():
    input_layer = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(64, (3,3), activation='relu')(input_layer)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    return models.Model(input_layer, x)

siamese_model = build_siamese_model()


In [6]:
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    return tf.reduce_mean(y_true * tf.square(y_pred) + (1 - y_true) * tf.square(tf.maximum(margin - y_pred, 0)))

siamese_model.compile(loss=contrastive_loss, optimizer='adam')

pairs, labels = [], []
for _ in range(10000):
    cls = np.random.choice(np.unique(y_train))
    idx = np.where(y_train == cls)[0]
    np.random.shuffle(idx)
    pairs.append([x_train[idx[0]], x_train[idx[1]]])
    labels.append(1)  # Positive pair

pairs, labels = np.array(pairs), np.array(labels)
siamese_model.fit([pairs[:, 0], pairs[:, 1]], labels, epochs=5)

# Evaluate on new pairs
test_pairs, test_labels = sample_episode(x_test, y_test, 2, 1, 1)
test_preds = siamese_model.predict([test_pairs[:, 0], test_pairs[:, 1]])
print("Siamese Network Results:")
print(classification_report(test_labels.flatten(), (test_preds > 0.5).astype(int).flatten()))


Epoch 1/5


ValueError: Layer "functional_1" expects 1 input(s), but it received 2 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(None, 28, 28, 1) dtype=float32>, <tf.Tensor 'data_1:0' shape=(None, 28, 28, 1) dtype=float32>]

In [10]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0,1]

# Reshape to (28,28,1)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Get classes
num_classes = 10
train_classes = {i: x_train[y_train == i] for i in range(num_classes)}
test_classes = {i: x_test[y_test == i] for i in range(num_classes)}


### **SIAMESE NETWORK**
def build_siamese_network(input_shape):
    input_1 = Input(input_shape)
    input_2 = Input(input_shape)

    def base_network(x):
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        return x

    embedding_1 = base_network(input_1)
    embedding_2 = base_network(input_2)

    # Distance function
    def euclidean_distance(vectors):
        return K.sqrt(K.sum(K.square(vectors[0] - vectors[1]), axis=1, keepdims=True))

    distance = Lambda(euclidean_distance)([embedding_1, embedding_2])

    model = Model(inputs=[input_1, input_2], outputs=distance)
    return model

siamese_model = build_siamese_network((28, 28, 1))
siamese_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Sample pairs
def sample_pairs(classes, num_pairs=500):
    pairs, labels = [], []
    
    for _ in range(num_pairs):
        label = random.choice([0, 1])
        cls = random.choice(list(classes.keys()))
        if label == 1:
            pair = np.random.choice(len(classes[cls]), 2, replace=False)
            pairs.append([classes[cls][pair[0]], classes[cls][pair[1]]])
        else:
            cls1, cls2 = np.random.choice(list(classes.keys()), 2, replace=False)
            pairs.append([random.choice(classes[cls1]), random.choice(classes[cls2])])

        labels.append(label)

    return np.array(pairs), np.array(labels)

train_pairs, train_labels = sample_pairs(train_classes, 500)
test_pairs, test_labels = sample_pairs(test_classes, 200)

# Train Siamese Network
siamese_model.fit([train_pairs[:, 0], train_pairs[:, 1]], train_labels, batch_size=16, epochs=5)

# Evaluate
pred_distances = siamese_model.predict([test_pairs[:, 0], test_pairs[:, 1]])
predictions = (pred_distances < 0.5).astype(int)
print("Siamese Network Results:")
print(classification_report(test_labels, predictions))

### **MATCHING NETWORK**
def cosine_similarity(vectors):
    x, y = vectors
    return K.sum(x * y, axis=1) / (K.sqrt(K.sum(K.square(x), axis=1)) * K.sqrt(K.sum(K.square(y), axis=1)))

def build_matching_network(input_shape):
    input_1 = Input(input_shape)
    input_2 = Input(input_shape)

    def base_network(x):
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        return x

    embedding_1 = base_network(input_1)
    embedding_2 = base_network(input_2)

    similarity = Lambda(cosine_similarity)([embedding_1, embedding_2])
    model = Model(inputs=[input_1, input_2], outputs=similarity)
    return model

matching_model = build_matching_network((28, 28, 1))
matching_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train Matching Network
matching_model.fit([train_pairs[:, 0], train_pairs[:, 1]], train_labels, batch_size=16, epochs=5)

# Evaluate
pred_similarities = matching_model.predict([test_pairs[:, 0], test_pairs[:, 1]])
predictions = (pred_similarities > 0.5).astype(int)
print("Matching Network Results:")
print(classification_report(test_labels, predictions))



Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.5103 - loss: 6.8001
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.4360 - loss: 8.9909
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.5178 - loss: 7.6876
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.5040 - loss: 7.9071
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.5103 - loss: 7.8074
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Siamese Network Results:
              precision    recall  f1-score   support

           0       0.42      1.00      0.60        85
           1       0.00      0.00      0.00       115

    accuracy                           0.42       200
   macro avg       0.21      0.50      0.30       200
weighted avg       0.18      0.42      0.25       200

Epoch 1/5


c:\Users\anand\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\anand\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\anand\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.0416 - loss: 0.6997
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.0234 - loss: 0.5273
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.0090 - loss: 0.2321
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.2643 - loss: 0.1125
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.2073 - loss: 0.0468
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Matching Network Results:
              precision    recall  f1-score   support

           0       0.66      0.81      0.73        85
           1       0.83      0.70      0.76       115

    accuracy                           0.74       200
   macro avg       0.75      0.75      0.74       200
weighted avg       0.76      0.74      0.75       200



In [9]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0,1]

# Reshape to (28,28,1)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Group images by class
num_classes = 10
train_classes = {i: x_train[y_train == i] for i in range(num_classes)}
test_classes = {i: x_test[y_test == i] for i in range(num_classes)}

# Sample few-shot episode
def sample_episode(classes, n_classes=5, k_shot=5, q_query=10):
    selected_classes = np.random.choice(list(classes.keys()), n_classes, replace=False)
    support_set, query_set, labels = [], [], []
    
    for label, cls in enumerate(selected_classes):
        samples = np.random.choice(len(classes[cls]), k_shot + q_query, replace=False)
        support, query = samples[:k_shot], samples[k_shot:]
        support_set.append(classes[cls][support])
        query_set.append(classes[cls][query])
        labels.append([label] * q_query)

    support_set = np.array(support_set)  # Shape: (N, k, 28, 28, 1)
    query_set = np.array(query_set)      # Shape: (N, q, 28, 28, 1)
    labels = np.array(labels).flatten()  # Shape: (N * q,)

    return support_set, query_set, labels

# Compute class prototypes (mean embedding)
def compute_prototypes(support_set):
    return np.mean(support_set, axis=1)  # Shape: (N, 28, 28, 1)

# Classify query samples based on Euclidean distance
def classify_query(query_set, prototypes):
    N, Q, H, W, C = query_set.shape  # N = num classes, Q = query samples/class

    # Reshape for broadcasting
    query_set = query_set.reshape(N * Q, H * W * C)  # (N*Q, 28*28*1)
    prototypes = prototypes.reshape(N, H * W * C)    # (N, 28*28*1)

    # Compute Euclidean distances
    distances = np.linalg.norm(query_set[:, np.newaxis, :] - prototypes, axis=2)  # (N*Q, N)
    
    return np.argmin(distances, axis=1)  # (N*Q,)

# Sample an episode
support, query, true_labels = sample_episode(train_classes)

# Compute prototypes & classify
prototypes = compute_prototypes(support)
pred_labels = classify_query(query, prototypes)

# Evaluate performance
print("Prototypical Network Results:")
print(classification_report(true_labels, pred_labels))


Prototypical Network Results:
              precision    recall  f1-score   support

           0       0.67      0.60      0.63        10
           1       0.60      0.30      0.40        10
           2       0.53      0.90      0.67        10
           3       0.62      0.50      0.56        10
           4       0.64      0.70      0.67        10

    accuracy                           0.60        50
   macro avg       0.61      0.60      0.58        50
weighted avg       0.61      0.60      0.58        50

